### Import libraries

In [1]:
import pandas as pd
import os
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_squared_error

### Set the data repository as the working directory 

In [2]:
print(os.path.abspath(os.curdir))

C:\Users\Asus\Documents\College\UP - MS EE\SY 2022-2023 Second Semester (temporary, final files in new laptop)\EE 290\Files (NEW Jun 22) FINAL FINAL\codes


In [3]:
os.chdir("../data")
print(os.path.abspath(os.curdir))

C:\Users\Asus\Documents\College\UP - MS EE\SY 2022-2023 Second Semester (temporary, final files in new laptop)\EE 290\Files (NEW Jun 22) FINAL FINAL\data


### Import the test set (solar and wind inclusion)

##### Test set (solar and wind inclusion)

In [4]:
pd_qd_test_solar_wind_45_pct = pd.read_csv('pd_qd_test_solar_wind_45_pct.csv')
pd_qd_test_solar_wind_45_pct.head()

,scenario,pd_2,qd_2,pd_3,qd_3,pd_4,qd_4,pd_5,qd_5,pd_6,...,pd_29,qd_29,pd_30,qd_30,pd_31,qd_31,pd_32,qd_32,pd_33,qd_33
0,1,0.034699,0.007573,0.045639,0.018038,0.028569,0.003493,0.031436,0.009529,0.027359,...,0.063776,0.040361,0.101603,0.058396,0.077677,0.030700,0.117351,0.086486,0.024336,0.002183
1,2,0.034556,0.007371,0.045064,0.017669,0.026869,0.003409,0.030837,0.009231,0.026833,...,0.062670,0.039661,0.099900,0.057418,0.076220,0.029884,0.115035,0.085078,0.024143,0.002166
2,3,0.034414,0.007166,0.044490,0.017302,0.022580,0.001942,0.030238,0.009051,0.026307,...,0.061563,0.039122,0.098197,0.056177,0.074763,0.029548,0.112719,0.083659,0.023950,0.002149
3,4,0.034272,0.006959,0.043915,0.017079,0.021249,0.001651,0.029639,0.008872,0.025781,...,0.060457,0.038577,0.096495,0.055202,0.073306,0.028741,0.110403,0.082227,0.023757,0.002131
4,5,0.034130,0.006750,0.043340,0.016992,0.019919,0.001060,0.029040,0.008693,0.025255,...,0.059350,0.037716,0.094792,0.054482,0.071848,0.028396,0.108087,0.080502,0.023564,0.002114


In [5]:
pg_v_test_solar_wind_45_pct = pd.read_csv('pg_v_test_solar_wind_45_pct.csv')
pg_v_test_solar_wind_45_pct.head()

,scenario,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,1,2.704067e-07,0.340860,1.022535,0.410543,1.004683,0.499993,1.001340,0.499991,1.016993
1,2,5.715829e-07,0.328133,1.022395,0.390855,1.004390,0.499982,1.001468,0.499973,1.017450
2,3,2.336102e-07,0.313061,1.022189,0.367603,1.004037,0.499991,1.001624,0.499984,1.017910
3,4,3.052111e-07,0.300664,1.022073,0.348467,1.003754,0.499974,1.001763,0.500000,1.018371
4,5,2.637911e-07,0.289214,1.021970,0.330980,1.003505,0.499958,1.001898,0.497436,1.018736


### Separate the dependent variables (optimal generator setpoints) and the independent variables (non-dispatchable loads)
Non-dispatchable loads: $x = (P_{D_{i}},Q_{D_{i}})\forall i\in \Omega_{D}$
<br>
Optimal generator setpoints: $y = (P_{G_{j}},V_{j})\forall j\in \Omega_{G}$

##### Test set (solar and wind inclusion)

In [6]:
x_test_solar_wind_45_pct = pd_qd_test_solar_wind_45_pct.loc[:, pd_qd_test_solar_wind_45_pct.columns != 'scenario']
x_test_solar_wind_45_pct.head()

,pd_2,qd_2,pd_3,qd_3,pd_4,qd_4,pd_5,qd_5,pd_6,qd_6,...,pd_29,qd_29,pd_30,qd_30,pd_31,qd_31,pd_32,qd_32,pd_33,qd_33
0,0.034699,0.007573,0.045639,0.018038,0.028569,0.003493,0.031436,0.009529,0.027359,0.019239,...,0.063776,0.040361,0.101603,0.058396,0.077677,0.030700,0.117351,0.086486,0.024336,0.002183
1,0.034556,0.007371,0.045064,0.017669,0.026869,0.003409,0.030837,0.009231,0.026833,0.018800,...,0.062670,0.039661,0.099900,0.057418,0.076220,0.029884,0.115035,0.085078,0.024143,0.002166
2,0.034414,0.007166,0.044490,0.017302,0.022580,0.001942,0.030238,0.009051,0.026307,0.018499,...,0.061563,0.039122,0.098197,0.056177,0.074763,0.029548,0.112719,0.083659,0.023950,0.002149
3,0.034272,0.006959,0.043915,0.017079,0.021249,0.001651,0.029639,0.008872,0.025781,0.018130,...,0.060457,0.038577,0.096495,0.055202,0.073306,0.028741,0.110403,0.082227,0.023757,0.002131
4,0.034130,0.006750,0.043340,0.016992,0.019919,0.001060,0.029040,0.008693,0.025255,0.017825,...,0.059350,0.037716,0.094792,0.054482,0.071848,0.028396,0.108087,0.080502,0.023564,0.002114


In [7]:
y_test_solar_wind_45_pct = pg_v_test_solar_wind_45_pct.loc[:, pg_v_test_solar_wind_45_pct.columns != 'scenario']
y_test_solar_wind_45_pct.head()

,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,2.704067e-07,0.340860,1.022535,0.410543,1.004683,0.499993,1.001340,0.499991,1.016993
1,5.715829e-07,0.328133,1.022395,0.390855,1.004390,0.499982,1.001468,0.499973,1.017450
2,2.336102e-07,0.313061,1.022189,0.367603,1.004037,0.499991,1.001624,0.499984,1.017910
3,3.052111e-07,0.300664,1.022073,0.348467,1.003754,0.499974,1.001763,0.500000,1.018371
4,2.637911e-07,0.289214,1.021970,0.330980,1.003505,0.499958,1.001898,0.497436,1.018736


### Test the trained models on the test set (solar and wind inclusion)

In [8]:
model_1 = load_model('model_1.h5')
model_2 = load_model('model_2.h5')
model_3 = load_model('model_3.h5')

##### Using model 1 that was trained using training group 1

In [9]:
y_test_solar_wind_45_pct_pred_1 = model_1.predict(x_test_solar_wind_45_pct)
y_test_solar_wind_45_pct_pred_1 = pd.DataFrame(data = y_test_solar_wind_45_pct_pred_1,
                                 columns = ['pg_1', 'pg_18', 'v_18', 'pg_22', 'v_22', 'pg_25', 'v_25', 'pg_33', 'v_33'])
y_test_solar_wind_45_pct_pred_1.head()

,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,0.000299,0.330947,1.023007,0.415204,1.004679,0.499998,1.000959,0.499951,1.017985
1,0.000292,0.318093,1.022867,0.395569,1.004388,0.500010,1.001083,0.499976,1.018447
2,0.000325,0.302421,1.022638,0.373072,1.004104,0.500025,1.001298,0.499421,1.019019
3,0.000301,0.292581,1.022508,0.357898,1.003958,0.500021,1.001509,0.495491,1.019393
4,0.000690,0.283495,1.022467,0.344606,1.003778,0.500007,1.001610,0.486383,1.019526


In [10]:
pg_v_test_solar_wind_45_pct_pred_1 = pd.concat([pg_v_test_solar_wind_45_pct.scenario, y_test_solar_wind_45_pct_pred_1], axis = 1)
pg_v_test_solar_wind_45_pct_pred_1.head()

,scenario,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,1,0.000299,0.330947,1.023007,0.415204,1.004679,0.499998,1.000959,0.499951,1.017985
1,2,0.000292,0.318093,1.022867,0.395569,1.004388,0.500010,1.001083,0.499976,1.018447
2,3,0.000325,0.302421,1.022638,0.373072,1.004104,0.500025,1.001298,0.499421,1.019019
3,4,0.000301,0.292581,1.022508,0.357898,1.003958,0.500021,1.001509,0.495491,1.019393
4,5,0.000690,0.283495,1.022467,0.344606,1.003778,0.500007,1.001610,0.486383,1.019526


In [11]:
pg_v_test_solar_wind_45_pct_pred_1.to_csv('pg_v_test_solar_wind_45_pct_pred_1.csv', index = False, float_format = '%0.20f')

##### Using model 2 that was trained using training group 2

In [12]:
y_test_solar_wind_45_pct_pred_2 = model_2.predict(x_test_solar_wind_45_pct)
y_test_solar_wind_45_pct_pred_2 = pd.DataFrame(data = y_test_solar_wind_45_pct_pred_2,
                             columns = ['pg_1', 'pg_18', 'v_18', 'pg_22', 'v_22', 'pg_25', 'v_25', 'pg_33', 'v_33'])
y_test_solar_wind_45_pct_pred_2.head()

,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,-0.000004,0.341297,1.022321,0.410918,1.004723,0.499839,1.001303,0.499769,1.016825
1,0.000003,0.328562,1.022171,0.391253,1.004440,0.499834,1.001425,0.499769,1.017270
2,0.000010,0.313477,1.021953,0.368063,1.004099,0.499826,1.001573,0.499768,1.017715
3,0.000016,0.301619,1.021823,0.349858,1.003849,0.499781,1.001693,0.498372,1.018089
4,0.000031,0.290284,1.021729,0.332555,1.003607,0.499621,1.001832,0.495326,1.018437


In [13]:
pg_v_test_solar_wind_45_pct_pred_2 = pd.concat([pg_v_test_solar_wind_45_pct.scenario, y_test_solar_wind_45_pct_pred_2], axis = 1)
pg_v_test_solar_wind_45_pct_pred_2.head()

,scenario,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,1,-0.000004,0.341297,1.022321,0.410918,1.004723,0.499839,1.001303,0.499769,1.016825
1,2,0.000003,0.328562,1.022171,0.391253,1.004440,0.499834,1.001425,0.499769,1.017270
2,3,0.000010,0.313477,1.021953,0.368063,1.004099,0.499826,1.001573,0.499768,1.017715
3,4,0.000016,0.301619,1.021823,0.349858,1.003849,0.499781,1.001693,0.498372,1.018089
4,5,0.000031,0.290284,1.021729,0.332555,1.003607,0.499621,1.001832,0.495326,1.018437


In [14]:
pg_v_test_solar_wind_45_pct_pred_2.to_csv('pg_v_test_solar_wind_45_pct_pred_2.csv', index = False, float_format = '%0.20f')

##### Using model 3 that was trained using training group 3

In [15]:
y_test_solar_wind_45_pct_pred_3 = model_3.predict(x_test_solar_wind_45_pct)
y_test_solar_wind_45_pct_pred_3 = pd.DataFrame(data = y_test_solar_wind_45_pct_pred_3,
                             columns = ['pg_1', 'pg_18', 'v_18', 'pg_22', 'v_22', 'pg_25', 'v_25', 'pg_33', 'v_33'])
y_test_solar_wind_45_pct_pred_3.head()

,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,0.000436,0.340470,1.022593,0.410382,1.004708,0.500102,1.001354,0.500296,1.017000
1,0.000452,0.327675,1.022459,0.390718,1.004415,0.500109,1.001487,0.500291,1.017457
2,0.000471,0.312537,1.022260,0.367513,1.004059,0.500116,1.001644,0.500284,1.017923
3,0.000484,0.300088,1.022175,0.348465,1.003776,0.500122,1.001766,0.500280,1.018407
4,0.000494,0.288835,1.022061,0.331449,1.003541,0.500137,1.001914,0.496956,1.018729


In [16]:
pg_v_test_solar_wind_45_pct_pred_3 = pd.concat([pg_v_test_solar_wind_45_pct.scenario, y_test_solar_wind_45_pct_pred_3], axis = 1)
pg_v_test_solar_wind_45_pct_pred_3.head()

,scenario,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,1,0.000436,0.340470,1.022593,0.410382,1.004708,0.500102,1.001354,0.500296,1.017000
1,2,0.000452,0.327675,1.022459,0.390718,1.004415,0.500109,1.001487,0.500291,1.017457
2,3,0.000471,0.312537,1.022260,0.367513,1.004059,0.500116,1.001644,0.500284,1.017923
3,4,0.000484,0.300088,1.022175,0.348465,1.003776,0.500122,1.001766,0.500280,1.018407
4,5,0.000494,0.288835,1.022061,0.331449,1.003541,0.500137,1.001914,0.496956,1.018729


In [17]:
pg_v_test_solar_wind_45_pct_pred_3.to_csv('pg_v_test_solar_wind_45_pct_pred_3.csv', index = False, float_format = '%0.20f')

##### RMSE of $P_{G_{1}}$ estimates

In [18]:
mean_squared_error(pg_v_test_solar_wind_45_pct.pg_1, pg_v_test_solar_wind_45_pct_pred_1.pg_1, squared = False)

0.004131739313693626

In [19]:
mean_squared_error(pg_v_test_solar_wind_45_pct.pg_1, pg_v_test_solar_wind_45_pct_pred_2.pg_1, squared = False)

0.0024346025167585068

In [20]:
mean_squared_error(pg_v_test_solar_wind_45_pct.pg_1, pg_v_test_solar_wind_45_pct_pred_3.pg_1, squared = False)

0.044988585037560265

##### RMSE of $P_{G_{18}}$ estimates

In [21]:
mean_squared_error(pg_v_test_solar_wind_45_pct.pg_18, pg_v_test_solar_wind_45_pct_pred_1.pg_18, squared = False)

0.0019372578269433493

In [22]:
mean_squared_error(pg_v_test_solar_wind_45_pct.pg_18, pg_v_test_solar_wind_45_pct_pred_2.pg_18, squared = False)

0.002803484457286958

In [23]:
mean_squared_error(pg_v_test_solar_wind_45_pct.pg_18, pg_v_test_solar_wind_45_pct_pred_3.pg_18, squared = False)

0.0025962518902024846

##### RMSE of $P_{G_{22}}$ estimates

In [24]:
mean_squared_error(pg_v_test_solar_wind_45_pct.pg_22, pg_v_test_solar_wind_45_pct_pred_1.pg_22, squared = False)

0.0024053124057540768

In [25]:
mean_squared_error(pg_v_test_solar_wind_45_pct.pg_22, pg_v_test_solar_wind_45_pct_pred_2.pg_22, squared = False)

0.00038056550690065223

In [26]:
mean_squared_error(pg_v_test_solar_wind_45_pct.pg_22, pg_v_test_solar_wind_45_pct_pred_3.pg_22, squared = False)

0.0026638203659590507

##### RMSE of $P_{G_{25}}$ estimates

In [27]:
mean_squared_error(pg_v_test_solar_wind_45_pct.pg_25, pg_v_test_solar_wind_45_pct_pred_1.pg_25, squared = False)

0.00010885776427362432

In [28]:
mean_squared_error(pg_v_test_solar_wind_45_pct.pg_25, pg_v_test_solar_wind_45_pct_pred_2.pg_25, squared = False)

0.0001536458038259203

In [29]:
mean_squared_error(pg_v_test_solar_wind_45_pct.pg_25, pg_v_test_solar_wind_45_pct_pred_3.pg_25, squared = False)

0.005661927172227479

##### RMSE of $P_{G_{33}}$ estimates

In [30]:
mean_squared_error(pg_v_test_solar_wind_45_pct.pg_33, pg_v_test_solar_wind_45_pct_pred_1.pg_33, squared = False)

0.0019803045268953524

In [31]:
mean_squared_error(pg_v_test_solar_wind_45_pct.pg_33, pg_v_test_solar_wind_45_pct_pred_2.pg_33, squared = False)

0.0003111619336120073

In [32]:
mean_squared_error(pg_v_test_solar_wind_45_pct.pg_33, pg_v_test_solar_wind_45_pct_pred_3.pg_33, squared = False)

0.0049178989639441315

##### RMSE of $V_{18}$ estimates

In [33]:
mean_squared_error(pg_v_test_solar_wind_45_pct.v_18, pg_v_test_solar_wind_45_pct_pred_1.v_18, squared = False)

0.0008491389191304605

In [34]:
mean_squared_error(pg_v_test_solar_wind_45_pct.v_18, pg_v_test_solar_wind_45_pct_pred_2.v_18, squared = False)

0.0005108811418438065

In [35]:
mean_squared_error(pg_v_test_solar_wind_45_pct.v_18, pg_v_test_solar_wind_45_pct_pred_3.v_18, squared = False)

0.005699819553599235

##### RMSE of $V_{22}$ estimates

In [36]:
mean_squared_error(pg_v_test_solar_wind_45_pct.v_22, pg_v_test_solar_wind_45_pct_pred_1.v_22, squared = False)

0.0003186210702204376

In [37]:
mean_squared_error(pg_v_test_solar_wind_45_pct.v_22, pg_v_test_solar_wind_45_pct_pred_2.v_22, squared = False)

0.0001616747330190258

In [38]:
mean_squared_error(pg_v_test_solar_wind_45_pct.v_22, pg_v_test_solar_wind_45_pct_pred_3.v_22, squared = False)

0.0025957279937287767

##### RMSE of $V_{25}$ estimates

In [39]:
mean_squared_error(pg_v_test_solar_wind_45_pct.v_25, pg_v_test_solar_wind_45_pct_pred_1.v_25, squared = False)

0.00041659205559242235

In [40]:
mean_squared_error(pg_v_test_solar_wind_45_pct.v_25, pg_v_test_solar_wind_45_pct_pred_2.v_25, squared = False)

0.00047222240592793586

In [41]:
mean_squared_error(pg_v_test_solar_wind_45_pct.v_25, pg_v_test_solar_wind_45_pct_pred_3.v_25, squared = False)

0.0064049104525385255

##### RMSE of $V_{33}$ estimates

In [42]:
mean_squared_error(pg_v_test_solar_wind_45_pct.v_33, pg_v_test_solar_wind_45_pct_pred_1.v_33, squared = False)

0.0007035616717975735

In [43]:
mean_squared_error(pg_v_test_solar_wind_45_pct.v_33, pg_v_test_solar_wind_45_pct_pred_2.v_33, squared = False)

0.00017792819454777104

In [44]:
mean_squared_error(pg_v_test_solar_wind_45_pct.v_33, pg_v_test_solar_wind_45_pct_pred_3.v_33, squared = False)

0.0012121992954921064